<a href="https://colab.research.google.com/github/LaboraDev/hackathon_one_flightOnTime/blob/main/DOC_DS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Relatório de Consolidação de Dados e ETL - Semana 01 (DS1)

****Detalhes do Projeto****
*   **Responsáveis:** Amélia e Ana, Time DS (Liderança: Helena)
*   **Repositório:** [Projeto FlightOnTime](https://github.com/LaboraDev/hackathon_one_flightOnTime/tree/main)
*   **Conformidade:** Dados públicos [ANAC](https://www.gov.br/anac/pt-br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/voo-regular-ativo-vra), anonimizados, em conformidade com a [LGPD](https://www.planalto.gov.br/ccivil_03/_ato2015-2018/2018/lei/l13709.htm) e Metodologia [D15](https://www.cirium.com/resources/on-time-performance/on-time-performance-faq/).
*   **Notebook Rastreável:** [S01_Consolidado_ETL_EDA.ipynb](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
*   **Eficiência Técnica:** Tipagem dos dados pensada para consumir pouca memória RAM.
*   **Status:** Finalizado | **Data:** 15/01/2026

---


## 1. Visão Geral

Durante a Semana 01, construímos o [pipeline](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb) de ingestão e tratamento dos dados da [ANAC](https://www.gov.br/anac/pt-br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/voo-regular-ativo-vra). O objetivo foi transformar os planos de voos do Brasil em uma [base de dados confiável](https://www.google.com/url?q=https%3A%2F%2Fdrive.google.com%2Ffile%2Fd%2F1K0QvUFmRkCM8-SrTdZEAp4ZNyOoL4j4V%2Fview%3Fusp%3Ddrive_link), usando a regra internacional [D15](https://www.cirium.com/resources/on-time-performance/on-time-performance-faq/) para marcar se um voo foi pontual ou não. Esse trabalho corrige erros antigos nos dados e cria a base necessária para ferramentas que preveem atrasos, ajudando a melhorar a viagem do passageiro e o trabalho das empresas. [FlightOnTime](https://github.com/LaboraDev/hackathon_one_flightOnTime/tree/main)


---

## 2. Dicionário de Terminologia Técnica
Para garantir a compreensão de todos, usamos as seguintes definições:
- **VRA:** Nome da lista oficial de voos da [ANAC](https://www.gov.br/anac/pt-br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/voo-regular-ativo-vra) que usamos como base
- **ICAO:** Código internacional de 4 letras que identifica aeroportos e empresas aéreas.
- **[D15](https://www.cirium.com/resources/on-time-performance/on-time-performance-faq/):** Padrão internacional de pontualidade da aviação que considera "pontual" voos com até 15 minutos de variação.
- **Target/Variável Alvo**: Variável que o modelo de Machine Learning tentará prever (situação de partida). [in [12]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
- **ETL:** Extract extração) [in [10]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb), Transform (transformação), Load (carregamento) de dados. [in [2]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
- **Dados Pessoais:** Dados sensíveis que identificam indivíduos (como CPF ou nome); não utilizamos nenhum dado sensível neste projeto por segurança.


---


## 3. [Dicionário de Variáveis](https://www.gov.br/anac/pt-br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/voo-regular-ativo-vra/62-voo-regular-ativo-vra) e Otimização de Memória
Para viabilizar o processamento de grandes volumes de dados no Google Colab, convertemos o formato de algumas informações (usando categorias e datas específicas). Garantindo performance e escalabilidade. [in [11]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)



| Nome da Variável | Tipo (Dtype) | Otimização | Descrição e Regra |
| :--- | :--- | :--- | :--- |
| `empresaaerea` | Category | Sim | Sigla ICAO da companhia aérea responsável pelo voo. |
| `numerovoo` | Object | - | Numeração identificadora do voo. |
| `aerodromoorigem` | Category | Sim | Sigla ICAO do aeroporto de partida. |
| `aerodromodestino` | Category | Sim | Sigla ICAO do aeroporto de chegada. |
| `partidaprevista` | Datetime64 | Sim | Horário programado para partida (Padrão Brasília/UTC-3). |
| `partidareal` | Datetime64 | Sim | Horário efetivo da decolagem registrado no log da ANAC. |
| `chegadaprevista` | Datetime64 | Sim | Horário programado para chegada. |
| `chegadareal` | Datetime64 | Sim | Horário efetivo da aterrissagem. |
| `situacaovoo` | Category | Sim | Status do voo (realizado, cancelado, não informado). |
| `situacaopartida` | **Target** | - | Rótulo de pontualidade baseado na regra D15 (variável alvo). |
| `codigojustificativa`| Nulo | - | Campo descontinuado pela ANAC pós-abr/2020 (100% nulo). |


---


## 4.2 Score de Confiabilidade por Dimensão

### 4.1 Funil de Processamento
| Etapa do Processamento | Volume de Registros | Impacto (%) | Justificativa Técnica |
| :--- | :--- | :--- | :--- |
| **Carga Bruta Inicial** | 3.968.418 | 100% | Total de voos registrados de 2020 a 2025. |
| **Filtro Período (2021-2025)** | 3.642.571 | -8,2% | Foco em anos com dados completos e relevantes. |
| **Remoção de Inconsistências** | 3.642.571 | < 0,1% | Exclusão de voos sem Aeródromo de Origem ou Partida Prevista. |
| **Dataset Final Útil** | **3.642.571** | **91,8%** | Base validada e pronta para análise exploratória e modelagem. |


Período Exato do Dataset Processado:
De 01/01/2021 00:05:00 até 01/07/2025 07:10:00.


### 4.2 Score de Confiabilidade por Dimensão
| Dimensão | Score | Observação Técnica |
| :--- | :--- | :--- |
| **Identificação** | 10/10 | Colunas `empresaaerea` e `numerovoo` com 100% de preenchimento. |
| **Temporalidade** | 9/10 | Precisão de segundos; necessitou remoção de logs com erro de sistema (>24h). |
| **Geografia** | 10/10 | Identificadores ICAO de origem e destino sem registros nulos. |
| **Causalidade** | 0/10 | Campo `codigojustificativa` descontinuado; impossível saber o "porquê" do atraso via dado bruto. |


---


## 5. Regras e decisões do projeto
Estabelecemos os seguintes pontos fundamentais para viabilizar a análise e garantir a integridade do pipeline:


*   **Processo de Ingestão:** Os dados brutos foram obtidos através do download individual dos arquivos mensais disponibilizados pela [ANAC](https://www.gov.br/anac/pt-br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/voo-regular-ativo-vra). Foi desenvolvido um [script de automação](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/script.py) no notebook para realizar a concatenação desses 54 arquivos, garantindo a continuidade histórica sem perda de registros entre os meses. [in [1][2][3]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
*   **Padrão de Tempo:** Todas as datas e horas seguem o fuso horário de Brasília (UTC-3, sem observância de horário de verão), garantindo a sincronia entre voos de diferentes regiões do país. [in [11]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
*   **On-Time Performance [(D15)](https://www.cirium.com/resources/on-time-performance/on-time-performance-faq/):** Adotamos o critério internacional da IATA, onde um voo é classificado como "Atrasado" apenas se a partida real exceder a prevista em mais de 15 minutos. Desvios menores são tratados como variações operacionais aceitáveis (Pontual).
*   **Filtro de Log (Sanidade):** Diferenças entre o horário previsto e real superiores a 24 horas são tratadas como erros de preenchimento ou falhas de sistema e removidas, pois não representam a realidade física da operação aérea. [in [12]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
*   **Dados Públicos e LGPD:** O dataset é composto exclusivamente por metadados operacionais de aeronaves e aeroportos extraídos de portal público. Não há coleta ou processamento de dados de passageiros, garantindo conformidade total com a Lei Geral de Proteção de Dados [(LGPD)](https://www.planalto.gov.br/ccivil_03/_ato2015-2018/2018/lei/l13709.htm)


---


## 6. Respostas às Perguntas-Chave


### 6.1 Qual é a origem pública dos dados?
Os dados provêm do portal de [Dados Abertos da ANAC (Agência Nacional de Aviação Civil)](https://www.gov.br/anac/pt-br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/voo-regular-ativo-vra), especificamente do conjunto denominado "Voo Regular Ativo (VRA)". Estes dados são públicos, anonimizados e disponibilizados livremente para fins de pesquisa e desenvolvimento. A equipe realizou o download via API de acesso público, garantindo conformidade com regulamentações de acesso à informação.


### 6.2 Qual problema real esses dados representam?
Os dados representam a **operação logística da malha aérea comercial brasileira**, detalhando o planejamento versus a execução real de voos. Especificamente, eles capturam a diferença entre o horário previsto e o horário executado de partidas e chegadas. Este fenômeno é crítico para compreender a eficiência operacional das companhias aéreas, a qualidade de serviço percebida pelos passageiros e os custos associados a atrasos (multas, compensações, impacto no itinerário subsequente).


### 6.3 O que cada linha do dataset representa?
Cada linha representa uma **etapa individual de um voo**. Se um voo faz escala (por exemplo, São Paulo → Rio de Janeiro → Brasília), são criadas duas linhas no dataset: uma para o trecho São Paulo-Rio e outra para Rio-Brasília. Esta granularidade permite análises detalhadas sobre gargalos específicos em rotas e horários.


### 6.4 Quais arquivos são carregados no projeto?
São carregados **54 arquivos CSV mensais** contidos no pacote comprimido [`dadosvra.zip`](https://www.google.com/url?q=https%3A%2F%2Fdrive.google.com%2Ffile%2Fd%2F1K0QvUFmRkCM8-SrTdZEAp4ZNyOoL4j4V%2Fview%3Fusp%3Ddrive_link). Cada arquivo corresponde a um mês do período de 2020 a 2025. A concatenação automática via Python garante que nenhum mês seja perdido e que a estrutura de colunas seja consistente. O total consolidado resulta em 3.968.418 registros. [in [1][2][3]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)


### 6.5 Quais colunas principais existem nos dados brutos?
As principais colunas são:
- **Operacionais:** `empresaaerea`, `numerovoo`, `aerodromoorigem`, `aerodromodestino`
- **Temporais:** `partidaprevista`, `partidareal`, `chegadaprevista`, `chegadareal`
- **Metadados:** `situacaovoo`, `codigojustificativa` (descontinuado), `codigoautorizacaodi`, `codigotipolinha`


Todas foram padronizadas para snake_case e tipadas corretamente para operações matemáticas e categóricas.


### 6.6 Quais tratamentos iniciais são aplicados?
- **Padronização de Nomes:** Conversão de colunas para `snake_case`, remoção de espaços e acentos, codificação uniforme (UTF-8). [in [9]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
- **Tipagem Temporal:** Conversão de strings para objetos `datetime64`, viabilizando cálculos de diferença temporal. [in [11]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
- **Tipagem Categórica:** Conversão de colunas repetidas (empresas, aeroportos) para tipo `category`, reduzindo footprint de memória em até 80%. [in [5]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
- **Validação de Estrutura:** Confirmação de que todos os 54 arquivos possuem a mesma estrutura de colunas antes da concatenação. [in [3]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)


### 6.7 Quais dados são descartados e por quê?
- **Período Fora de 2021-2025:** Registros anteriores a 2021 foram excluídos por apresentarem inconsistências históricas nas bases da [ANAC](https://www.gov.br/anac/pt-br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/voo-regular-ativo-vra). Registros após 2025 não existem.
- **Outliers Temporais Críticos:** Voos com diferença > 24 horas entre previsto e real foram sinalizados e removidos, pois indicam **erros de preenchimento de log** (ex: voo com data de partida futura ou atraso impossível fisicamente). [in [12]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
- **Campos Obrigatórios Nulos:** Registros sem `aerodromoorigem` ou `partidaprevista` foram removidos, pois não permitem o cálculo do target. [in [24]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)


Esta abordagem preserva a integridade dos dados e evita que o modelo aprenda com ruído (GIGO - *Garbage In, Garbage Out*).


### 6.8 Como a variável alvo "atrasado" é definida?
A variável alvo `situacaopartida` [in [12]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb) é calculada a partir do **delta temporal**:
$$\Delta
 = \text{Partida Real} - \text{Partida Prevista}$$


O resultado (em minutos) é então classificado conforme a regra de negócio [D15 (padrão IATA)](https://www.cirium.com/resources/on-time-performance/on-time-performance-faq/) :
- **Antecipado:** \( \Delta < 0 \) min (partida antes do programado)
- **Pontual:** \( 0 \leq \Delta \leq 15 \) min (dentro da tolerância de 15 minutos)
- **Atraso 15-60:** \( 15 < \Delta \leq 60 \) min
- **Atraso 60-120:** \( 60 < \Delta \leq 120 \) min
- **Atraso 120-240:** \( 120 < \Delta \leq 240 \) min
- **Atraso > 240:** \( \Delta > 240 \) min


A distribuição final é:
- Antecipado: 55,40%
- Pontual: 28,62%
- Atraso 15-60: 12,52%
- Atraso 60-120: 2,14%
- Atraso 120-240: 0,91%
- Atraso > 240: 0,41%


### 6.9 Quais limitações já existem nos dados antes da modelagem? [in [25]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb)
- **Coluna Descontinuada:** A coluna `codigojustificativa` (que deveria explicar o motivo do atraso) é 100% nula a partir de abril de 2020. Isso significa que o modelo não terá acesso à informação causal e dependerá exclusivamente de padrões temporais e geográficos.
- **Desbalanceamento Severo:** A classe majoritária (Antecipado) representa 55,4% dos dados, enquanto a classe mais rara (Atraso > 240 min) representa apenas 0,41%. Este desbalanceamento pode gerar viés do modelo para a classe majoritária, exigindo técnicas de balanceamento (SMOTE, pesos de classe, undersampling) na fase de treinamento.
- **Ausência de Contexto Externo:** O dataset não contém variáveis externas como clima, volume de passageiros, combustível ou manutenções não planejadas, que são fatores reais de atrasos.


---


## 7. Análise de Riscos para Modelagem


| Risco | Severidade | Mitigation Strategy |
| :--- | :--- | :--- |
| **Desbalanceamento de Classes** | Alta | Aplicar SMOTE ou pesos de classe durante o treinamento. |
| **Falta de Causalidade** | Média | Focar em feature engineering temporal e geográfico. |
| **Outliers Estruturais** | Baixa | Já tratados na fase de ETL (remoção de discrepâncias > 24h). |
| **Data Leakage** | Média | Garantir que features de chegada não sejam usadas para prever partida. |


---


## 8. Checklist de Integridade (Selo de Qualidade)
- [x] **Segurança:** Documentação e processo alinhados à LGPD (dados anonimizados, públicos).
- [x] **Performance:** Redução de footprint de memória via tipos `category` (teste: carregamento bem-sucedido no Google Colab com <2GB).
- [x] **Consistência:** Remoção de outliers temporais e dados fora do período de estudo (2021-2025).
- [x] **Rastreabilidade:** Mapeamento direto entre narrativa e células de código do notebook.
- [x] **Reprodutibilidade:** Pipeline automatizado; qualquer desenvolvedor pode reproduzir em 5 minutos.


---


## 9. Blueprint de Rastreabilidade
Para auditoria técnica e validação, utilize as referências abaixo no notebook [`S01_Consolidado_ETL_EDA.ipynb`](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana01/S01_Consolidado_ETL_EDA.ipynb) :


| Etapa | Células | Descrição |
| :--- | :--- | :--- |
| **Extração e Download** | [1] - [3] | Instalação de dependências (gdown, pandas) e download do ZIP. |
| **Validação de Arquivos** | [4] - [6] | Verificação de que 54 arquivos existem e contêm o volume esperado. |
| **Padronização de Schema** | [7] - [9] | Renomeação de colunas, decodificação Latin-1 para UTF-8. |
| **Engenharia de Flags** | [11] - [12] | Criação de colunas booleanas para auditoria (datas ausentes, outliers). |
| **Lógica do Target (D15)** | [14] - [17] | Função de classificação e aplicação da variável alvo. |
| **Visualização e Exportação** | [18] - [19] | Gráfico de distribuição do target e exportação de `dataset_estruturado.csv`. |


---


## 10. Conclusão


### Validação da Semana 01
O pipeline de ETL foi consolidado com sucesso. O dataset estruturado gerado possui:
- **91,8% de retenção de dados** (3.642.571 de 3.968.418 registros originais).
- **Nenhuma violação de privacidade** (conformidade LGPD).
- **Tipagem otimizada para produção** (consumo controlado de memória).

---


## Referências
- ANAC - Agência Nacional de Aviação Civil. (2024). Portal de Dados Abertos - VRA. Disponível em: [ANAC](https://www.gov.br/anac/pt-br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/voo-regular-ativo-vra)
- IATA - International Air Transport Association. (2023). D15 Punctuality Standard. Disponível em: [D15 (padrão IATA)](https://www.cirium.com/resources/on-time-performance/on-time-performance-faq/)
- Lei Geral de Proteção de Dados (LGPD). Lei nº 13.709, de 14 de agosto de 2018. Brasil. Disponível em: [LGPD](https://www.planalto.gov.br/ccivil_03/_ato2015-2018/2018/lei/l13709.htm)
- Disponibilidade do Dataset: O arquivo final, consolidado e processado pela equipe, pode ser acessado em: [dadosvra.zip](https://www.google.com/url?q=https%3A%2F%2Fdrive.google.com%2Ffile%2Fd%2F1K0QvUFmRkCM8-SrTdZEAp4ZNyOoL4j4V%2Fview%3Fusp%3Ddrive_link)